In [ ]:
! pip install flask==0.12.2
! pip install flask-ngrok
! pip install pyngrok
! pip install av

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv

from flask import Flask, jsonify, request, render_template
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok

from google.colab import files

import cv2

import os
from collections import OrderedDict

In [ ]:
with open('ngrok_token.txt', 'r') as f:
  ngrok_token = f.readlines()[0]

In [ ]:

class SpatialAttention(nn.Module):

  def __init__(self):
    super(SpatialAttention, self).__init__()
  
  def forward(self, x_q, x_k, x_v):

    b, c, t, h, w = x_q.size()

    r_q = x_q.permute((0, 2, 1, 3, 4)).reshape(b*t, c, h*w)
    r_k = x_k.permute((0, 2, 1, 3, 4)).reshape(b*t, c, h*w)
    r_v = x_v.permute((0, 2, 1, 3, 4)).reshape(b*t, c, h*w)

    m_c = T.matmul(r_q, r_k.transpose(1, 2))
    m_c = F.softmax(m_c, dim=0)
    a_c = T.matmul(m_c, r_v)

    m_s = T.matmul(r_q.transpose(1, 2), r_k)
    m_s = F.softmax(m_s, dim=0)
    a_s = T.matmul(m_s, r_v.transpose(1, 2)).transpose(1, 2)

    a = a_c + a_s
    a = a.reshape(b, t, c, h, w).permute((0, 2, 1, 3, 4))
    return a


class TemporalAttention(nn.Module):

  def __init__(self):
    super(TemporalAttention, self).__init__()
  
  def forward(self, x, x_v):

    b, c, t, h, w = x.size()

    r_q = x.permute((0, 2, 1, 3, 4)).reshape(b, t, c*h*w)
    r_k = r_q.transpose(1, 2)
    r_v = x_v.permute((0, 2, 1, 3, 4)).reshape(b, t, c*h*w)

    m_t = T.matmul(r_q, r_k)
    m_t = F.softmax(m_t, dim=0)
    a_t = T.matmul(m_t, r_v)

    a = a_t.reshape(b, t, c, h, w).permute((0, 2, 1, 3, 4))
    return a


class SSA(nn.Module):

  def __init__(self, in_dim, out_dim):
    super(SSA, self).__init__()

    self.x_q_embed = nn.Conv3d(in_dim, out_dim, kernel_size=(1, 1, 1))
    self.x_k_embed = nn.Conv3d(in_dim, out_dim, kernel_size=(1, 1, 1))
    self.x_v_embed = nn.Conv3d(in_dim, out_dim, kernel_size=(1, 1, 1))
    self.spatial = SpatialAttention()

    self.X_embed = nn.Conv3d(in_dim, out_dim, kernel_size=(3, 1, 1), padding=(1, 0, 0))
    self.temporal = TemporalAttention()
  
  def forward(self, x):
    '''
      x = T.tensor of shape (B, C, T, H, W)
    '''

    x_q = F.relu(self.x_q_embed(x))
    x_k = F.relu(self.x_k_embed(x))
    x_v = F.relu(self.x_v_embed(x))
    s = self.spatial(x_q, x_k, x_v)

    X = F.relu(self.X_embed(s))
    t = self.temporal(X, x_v)

    return t


In [ ]:

class IdBlock(nn.Module):

  def __init__(self, in_dim, hidden_dim, kernel_size):
    super(IdBlock, self).__init__()

    self.conv1 = nn.Conv3d(in_dim, hidden_dim, 1)
    self.norm1 = nn.InstanceNorm3d(hidden_dim)

    self.conv2 = nn.Conv3d(hidden_dim, hidden_dim, kernel_size=kernel_size, padding='same')
    self.norm2 = nn.InstanceNorm3d(hidden_dim)

    self.conv3 = nn.Conv3d(hidden_dim, in_dim, 1)
    self.norm3 = nn.InstanceNorm3d(in_dim)

  def forward(self, x):

    x_shortcut = x

    x = self.conv1(x)
    x = self.norm1(x)
    x = F.relu(x)

    x = self.conv2(x)
    x = self.norm2(x)
    x = F.relu(x)

    x = self.conv3(x)
    x = self.norm3(x)

    x = x + x_shortcut
    x = F.relu(x)
    
    return x


class ConvBlock(nn.Module):

  def __init__(self, in_dim, hidden_dim, out_dim, kernel_size, stride=1):
    super(ConvBlock, self).__init__()

    self.conv0 = nn.Conv3d(in_dim, out_dim, 1, stride=stride)
    self.norm0 = nn.InstanceNorm3d(out_dim)

    self.conv1 = nn.Conv3d(in_dim, hidden_dim, 1, stride=stride)
    self.norm1 = nn.InstanceNorm3d(hidden_dim)

    self.conv2 = nn.Conv3d(hidden_dim, hidden_dim, kernel_size=kernel_size, padding='same')
    self.norm2 = nn.InstanceNorm3d(hidden_dim)

    self.conv3 = nn.Conv3d(hidden_dim, out_dim, 1)
    self.norm3 = nn.InstanceNorm3d(out_dim)
  
  def forward(self, x):

    x_shortcut = self.conv0(x)
    x_shortcut = self.norm0(x_shortcut)

    x = self.conv1(x)
    x = self.norm1(x)
    x = F.relu(x)

    x = self.conv2(x)
    x = self.norm2(x)
    x = F.relu(x)

    x = self.conv3(x)
    x = self.norm3(x)

    x = x + x_shortcut
    x = F.relu(x)
    
    return x


class SSABlock(nn.Module):

  def __init__(self, in_dim, hidden_dim, kernel_size):
    super(SSABlock, self).__init__()

    self.conv1 = nn.Conv3d(in_dim, hidden_dim, 1)
    self.norm1 = nn.InstanceNorm3d(hidden_dim)

    self.ssa2 = SSA(hidden_dim, hidden_dim)

    self.conv3 = nn.Conv3d(hidden_dim, hidden_dim, kernel_size=kernel_size, padding='same')
    self.norm3 = nn.InstanceNorm3d(hidden_dim)

    self.conv4 = nn.Conv3d(hidden_dim, in_dim, 1)
    self.norm4 = nn.InstanceNorm3d(in_dim)

  def forward(self, x):

    x_shortcut = x

    x = self.conv1(x)
    x = self.norm1(x)
    x = F.relu(x)

    x = self.ssa2(x)

    x = self.conv3(x)
    x = self.norm3(x)
    x = F.relu(x)

    x = self.conv4(x)
    x = self.norm4(x)

    x = x + x_shortcut
    x = F.relu(x)
    
    return x
  

In [ ]:

class Model(nn.Module):

  def __init__(self):
    super(Model, self).__init__()

    self.conv1 = nn.Sequential(OrderedDict([
        ('conv1a', nn.Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3))),
        ('conv1b', nn.InstanceNorm3d(64)),
        ('conv1c', nn.ReLU()),
        ('conv1d', nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1))),
    ]))

    self.res2 = nn.Sequential(OrderedDict([
        ('res2a', ConvBlock(64, 64, 256, kernel_size=(1, 3, 3))),
        ('res2b', SSABlock(256, 64, kernel_size=(1, 3, 3))),
        ('res2c', SSABlock(256, 64, kernel_size=(1, 3, 3))),
    ]))

    self.res3 = nn.Sequential(OrderedDict([
        ('res3a', ConvBlock(256, 128, 512, kernel_size=(1, 3, 3), stride=(1, 2, 2))),
        ('res3b', SSABlock(512, 128, kernel_size=(1, 3, 3))),
        ('res3c', SSABlock(512, 128, kernel_size=(1, 3, 3))),
        ('res3d', SSABlock(512, 128, kernel_size=(1, 3, 3))),
    ]))

    self.res4 = nn.Sequential(OrderedDict([
        ('res4a', ConvBlock(512, 256, 1024, kernel_size=(3, 3, 3), stride=(1, 2, 2))),
        ('res4b', IdBlock(1024, 256, kernel_size=(3, 3, 3))),
        ('res4c', IdBlock(1024, 256, kernel_size=(3, 3, 3))),
        ('res4d', IdBlock(1024, 256, kernel_size=(3, 3, 3))),
        ('res4e', IdBlock(1024, 256, kernel_size=(3, 3, 3))),
        ('res4f', IdBlock(1024, 256, kernel_size=(3, 3, 3))),
    ]))

    self.res5 = nn.Sequential(OrderedDict([
        ('res5a', ConvBlock(1024, 512, 2048, kernel_size=(3, 3, 3), stride=(2, 2, 2))),
        ('res5b', IdBlock(2048, 512, kernel_size=(3, 3, 3))),
        ('res5c', IdBlock(2048, 512, kernel_size=(3, 3, 3))),
    ]))

    self.fc6 = nn.Sequential(OrderedDict([
        ('fc6a', nn.AvgPool3d(kernel_size=(1, 7, 7))),
        ('fc6b', nn.Flatten(start_dim=1)),
        ('fc6c', nn.Dropout(p=0.3)),
        ('fc6d', nn.Linear(8192, 1024)),
        ('fc6e', nn.ReLU()),
        ('fc6f', nn.Dropout(p=0.3)),
        ('fc6g', nn.Linear(1024, 1)),
    ]))
    
  
  def forward(self, x):

    x = self.conv1(x)
    x = self.res2(x)
    x = self.res3(x)
    x = self.res4(x)
    x = self.res5(x)
    x = self.fc6(x)

    return x



In [ ]:
model_state = T.load('model')


In [ ]:

device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
model_state = T.load('model')
model = Model()
model.load_state_dict(model_state)
model = model.to(device).eval()

transform = tv.transforms.Compose([
  tv.transforms.CenterCrop((720, 720)).to(device),
  tv.transforms.Resize((224, 224)).to(device),
  tv.transforms.ConvertImageDtype(T.float).to(device)
])

raft_weights = tv.models.optical_flow.Raft_Large_Weights.DEFAULT
raft_transforms = raft_weights.transforms().to(device)
raft = tv.models.optical_flow.raft_large(weights=raft_weights, progress=False)
raft = raft.to(device).eval()

def optical_flow(frames1, frames2):
  frames1 = frames1.to(device)
  frames2 = frames2.to(device)
  t_list = list()
  with T.no_grad():
    for img1, img2 in zip(frames1, frames2):
      img1 = T.unsqueeze(img1, 0)
      img2 = T.unsqueeze(img2, 0)
      img1, img2 = raft_transforms(img1, img2)
      t_list.append(raft(img1, img2)[-1])
  return T.squeeze(T.stack(t_list))

grayscale = tv.transforms.Grayscale().to(device)

normalize = tv.transforms.Normalize(
  T.tensor([0.3002, -0.0349, -0.0826]), 
  T.tensor([0.2623, 4.1885, 2.1467])
).to(device)


In [ ]:
app = Flask(__name__)
ngrok.set_auth_token(ngrok_token)
run_with_ngrok(app)

def transform_video(video_path, frame_freq):
  vid, _, _ = tv.io.read_video(video_path)
  vid = T.permute(vid, (0, 3, 2, 1))
  vid = transform(vid).to(device)
  frame_idx_list = list(range(16, len(vid), frame_freq))
  tensor_list = list()
  for idx in frame_idx_list:
    frames1 = vid[idx-16: idx: 2, :, :, :]
    frames2 = vid[idx-15: idx+1: 2, :, :, :]
    flow = optical_flow(frames1, frames2)
    gray = grayscale(frames1)
    output = T.cat([gray, flow], dim=1)
    output = normalize(output)
    output = T.permute(output, (1, 0, 2, 3)).unsqueeze(0)
    output = output.to(device)
    tensor_list.append((idx, output))
  return tensor_list


def get_prediction(video_path, frame_freq):
  tensor_list = transform_video(video_path, frame_freq)
  with T.no_grad():
    pred_dict = {idx: model(tensor).item() for idx, tensor in tensor_list}
  return pred_dict


def write_prediction(pred_dict, in_path, out_path):
  cap = cv2.VideoCapture(in_path)
  frame_width = int(cap.get(3))
  frame_height = int(cap.get(4)) 
  out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'VP80'),
                        30, (frame_width, frame_height))
  frame_idx = 0
  while(True):
    ret, frame = cap.read()
    if ret == True:
      pred_idx = frame_idx // 16 * 16
      text = pred_dict.get(pred_idx)
      if text is None:
        text = pred_dict.get(pred_idx + 16)
      if text is None:
        text = pred_dict.get(pred_idx - 16)
      cv2.putText(frame, f'{text:.1f}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
      out.write(frame)
      frame_idx += 1
    else:
      break

  cap.release()
  out.release()


@app.route('/')
def home():
  return render_template('index.html')


@app.route('/predict', methods=['POST'])
def predict():
  if request.method == 'POST':
    if 'in_file' in os.listdir('static'):
      os.remove(os.path.join('static', 'in_file'))
    if 'out_file.webm' in os.listdir('static'):
      os.remove(os.path.join('static', 'out_file.webm'))
    file_upload = request.files['file']
    file_upload.save(os.path.join('static', 'in_file'))
    pred_dict = get_prediction(os.path.join('static', 'in_file'), 16)
    write_prediction(pred_dict, os.path.join('static', 'in_file'), os.path.join('static', 'out_file.webm'))
    return render_template('index.html', filename='out_file.webm')

app.run()

INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://0d0f-35-204-12-215.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [22/Aug/2022 15:45:46] "GET / HTTP/1.1" 200 -
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:162: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")
INFO:werkzeug:127.0.0.1 - - [22/Aug/2022 15:49:03] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Aug/2022 15:49:04] "GET /static/out_file.webm HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [22/Aug/2022 15:49:12] "GET /static/out_file.webm HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [22/Aug/2022 15:49:13] "GET /static/out_file.webm HTTP/1.1" 206 -
